# Project Description:

The data and the objective of the project was provided by [Udacity](https://udacity.com) as part of the course A/B Testing.

The task was to perform an A/B Test for the given objective and data.

Udacity students have two opitons to enroll for a course "start free trial", and "access course materials". If the student clicks "start free trial", they will be asked to enter their credit card information, and then they will be enrolled in a free trial for the paid version of the course. After 14 days, they will automatically be charged unless they cancel first. If the student clicks "access course materials", they will be able to view the videos and take the quizzes for free, but they will not receive coaching support or a verified certificate, and they will not submit their final project for feedback. 

## Point of Analyses

Udacity tested a change where if the student clicked "start free trial", they were asked how much time they had available to devote to the course. If the student indicated 5 or more hours per week, they would be taken through the checkout process as usual. If they indicated fewer than 5 hours per week, a message would appear indicating that Udacity courses usually require a greater time commitment for successful completion, and suggesting that the student might like to access the course materials for free. At this point, the student would have the option to continue enrolling in the free trial, or access the course materials for free instead.

Hypothesis:

The change might set clearer expectations for students upfront, thus reducing the number of frustrated students who left the free trial because they didn't have enough time—without significantly reducing the number of students to continue past the free trial and eventually complete the course. If this hypothesis held true, Udacity could improve the overall student experience and improve coaches' capacity to support students who are likely to complete the course.

The unit of diversion is a cookie, although if the student enrolls in the free trial, they are tracked by user-id from that point forward. The same user-id cannot enroll in the free trial twice. For users that do not enroll, their user-id is not tracked in the experiment, even if they were signed in when they visited the course overview page.

# Project Preperations

## Import Libaries

In [1]:
import sys
import math

from IPython.display import display

import pandas as pd

from scipy.stats import norm, binom_test, fisher_exact, sem
from termcolor import colored, cprint

## Set Global Variables

In [2]:
### Path to the baseline values data
BASE_PATH = 'data/Final Project Baseline Values.xlsx'

### Path to the experiment and controll data
RESULTS_PATH = 'data/Final Project Results.xlsx'

## Load the Data

### Base Values

In [3]:
### Load Baseline Values
df_base = pd.read_excel(BASE_PATH, header = None)

### Set the header
df_base.columns = ['Metric', 'Value']
### Set the index to the metrics name
df_base.index = list(df_base['Metric'])

### Display Baseline Values
display(df_base)

,Metric,Value
Unique cookies to view page per day:,Unique cookies to view page per day:,40000.000000
"Unique cookies to click ""Start free trial"" per day:","Unique cookies to click ""Start free trial"" per...",3200.000000
Enrollments per day:,Enrollments per day:,660.000000
"Click-through-probability on ""Start free trial"":","Click-through-probability on ""Start free trial"":",0.080000
"Probability of enrolling, given click:","Probability of enrolling, given click:",0.206250
"Probability of payment, given enroll:","Probability of payment, given enroll:",0.530000
"Probability of payment, given click","Probability of payment, given click",0.109313


### Experiment Data

#### Controll group data

In [4]:
### Load experiment controll group data 
df_control = pd.read_excel(RESULTS_PATH, sheetname = 0)

### Controll group data
display(df_control.head())

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7723,687,134.0,70.0
1,"Sun, Oct 12",9102,779,147.0,70.0
2,"Mon, Oct 13",10511,909,167.0,95.0
3,"Tue, Oct 14",9871,836,156.0,105.0
4,"Wed, Oct 15",10014,837,163.0,64.0


#### Controll experiment data

In [5]:
### Load experiment experiment group data 
df_exp = pd.read_excel(RESULTS_PATH, sheetname = 1)

### Controll experiment data
display(df_exp.head())

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7716,686,105.0,34.0
1,"Sun, Oct 12",9288,785,116.0,91.0
2,"Mon, Oct 13",10480,884,145.0,79.0
3,"Tue, Oct 14",9867,827,138.0,92.0
4,"Wed, Oct 15",9793,832,140.0,94.0


# Perform A/B Test

## Metric Choice

### Invariant Metrics

*  <span style="color:blue">**Number of cookies:** That is, number of unique cookies to view the course overview page.</span> 
*  <span style="color:blue">**Number of clicks:** That is, number of unique cookies to click the "Start free trial" button (which happens before the free trial screener is trigger).</span> 
*  <span style="color:blue">**Click-through-probability:** That is, number of unique cookies to click the "Start free trial" button divided by number of unique cookies to view the course overview page.</span> 


##### Rationale

**Unit of diversion: Cookie**

* **Number of cookies:** As it is the unit of diversion, it is ranomly assigned the experiment and controll group.
* **Number of user-ids:** Since it happens after the free trial screener is triggered the number of user-ids should be affected by the change.
* **Number of clicks:** Since it happens before the free trial screener is triggered the number of clicks shouldn't be affected by the change. They are not directly randomized but split evenly between the groups.
* **Click-through-probability:** Since it happens before the free trial screener is triggered the CTP shouldn't be affected by the change.
* **Gross conversion|Retention|Net conversion:** All of these depends on the user-ids that are affected by the change.



### Evaluation Metrics


*  <span style="color:blue">**Gross conversion:** That is, number of user-ids to complete checkout and enroll in the free trial divided by number of unique cookies to click the "Start free trial" button. </span>
*  <span style="color:blue">**Retention:** That is, number of user-ids to remain enrolled past the 14-day boundary (and thus make at least one payment) divided by number of user-ids to complete checkout.</span> 
*  <span style="color:blue">**Net conversion:** That is, number of user-ids to remain enrolled past the 14-day boundary (and thus make at least one payment) divided by the number of unique cookies to click the "Start free trial" button.</span> 


##### Rationale

* **Gross conversion:**  Used as evalution metric to evalute if the number of user-ids to complete checkout and enroll in the free trial divided by number of unique cookies to click the "Start free trial" button changes. Expected results: Decreases because less people should decide for the free trial.
* **Retention:** Not used as an evalution metric because it would need a to big sample size to be releveant. 
* **Net conversion:** Used as evalution metric to evalute if the number of user-ids to remain enrolled past the 14-day boundary divided by the number of unique cookies to click the "Start free trial" button changes. Expected results: Stays on the same level or increase.

##### Rationale for ignoring number of user-ids

Since th user-ids are collected after the free trial screener is triggered the metric is affeted by the change and therefore, it would be no good invariant metric. 

It is not used as evalution metric as it represents the raw count of users who enroll in the free trial, that is related to the metric gross conversion. Since both encodes similar information we keep that metric that performs better on the experiment setup.   
The reason for this is that the gross conversion is able to normalize between different sized experiment and control groups, because it uses the number of cookies as denominator. Therfore the metric will adjust to different number of cookies between the experiment and controll group and will be more accurate than the raw raw count of users-ids.

## Measuring Variability

### Analytical estimate of the standard deviation

### Compute baseline values for 5000 pageviews

In [6]:
### Set number of samples
sampleSize = 5000

### Compute 
factor = df_base['Value']['Unique cookies to view page per day:'] / sampleSize

### Compute baseline values for 5000 pageviews
df_base['ValueSample'] = df_base['Value'] / factor

### Print the baseline values for the number of samples
display(df_base)

,Metric,Value,ValueSample
Unique cookies to view page per day:,Unique cookies to view page per day:,40000.000000,5000.000000
"Unique cookies to click ""Start free trial"" per day:","Unique cookies to click ""Start free trial"" per...",3200.000000,400.000000
Enrollments per day:,Enrollments per day:,660.000000,82.500000
"Click-through-probability on ""Start free trial"":","Click-through-probability on ""Start free trial"":",0.080000,0.010000
"Probability of enrolling, given click:","Probability of enrolling, given click:",0.206250,0.025781
"Probability of payment, given enroll:","Probability of payment, given enroll:",0.530000,0.066250
"Probability of payment, given click","Probability of payment, given click",0.109313,0.013664


### Compute analytic estimates of variance

In [7]:
def calc_std(p, N):
    """ Computes the analytic estimate of variance (std) using the formuala:
       p(1-p) / n

    Args:
        p, probaility
        N, number of samples

    Returns:
        std
    """
    return math.sqrt(p* (1-p) / N)


### Compute the std for probability of enrolling, given click:
std_enrolling = calc_std(df_base['Value']['Probability of enrolling, given click:'],
                         df_base['ValueSample']['Unique cookies to click "Start free trial" per day:'])

### Compute the std for probability of payment, given enroll:
std_payment_enroll = calc_std(df_base['Value']['Probability of payment, given enroll:'], 
                              df_base['ValueSample']['Enrollments per day:'])

### Compute the std for probability of payment, given click
std_payment_click = calc_std(df_base['Value']['Probability of payment, given click'], 
                             df_base['ValueSample']['Unique cookies to click "Start free trial" per day:'])


### Print the std for the evalution metrics
print(colored("Standard deviation(SE) gross conversion: " +
              str(round(std_enrolling,4)), 'blue') + 
              ' (accurate)')
print(colored("Standard deviation(SE) retention: " +
              str(round(std_payment_enroll,4)), 'blue') + 
              ' (accurate)')
print(colored("Standard deviation(SE) net conversion: " +
              str(round(std_payment_click,4)), 'blue') + 
              ' (accurate)')

Standard deviation(SE) gross conversion: 0.0202 (accurate)
Standard deviation(SE) retention: 0.0549 (accurate)
Standard deviation(SE) net conversion: 0.0156 (accurate)


#### Rationale



For the metrics we can look if the unit of diversion and the unit of the evalution metric mates. If this is the case the analytic estimate  can be used and there is no need to use the empirical estimate.

*  Standard deviation(SE) gross conversion: As the unit of diversion is cookies and gross conversion is defined as "The number of user-ids to complete checkout and enroll in the free trial divided by number of unique cookies to click the "Start free trial" button", the unit of denominator of the fraction is also cookies. Because both the unit of diversion and the unit denominator are the same the analytic estimates can be used and it is accurate.

*  Standard deviation(SE) retention: As he unit of diversion is cookies and retention is defined as " The number of user-ids to remain enrolled past the 14-day boundary divided by number of user-ids to complete checkout", the unit of denominator of the fraction is the user-id. Both does not exactly match, but as the user-id should be nearly identical to the cookie expect in cases there the user switches the devices, the browser or removes the cookies. It the unit of the unit of diversion and the unit of denominator of the fraction should almost match. Therefore, as the unit of diversion and the unit denominator are the nearly the same unit the analytic estimates should be mostly accurate, but as it is not exactly the same if there is time further data should become be collected to have a mor accurate estimate.

*  Standard deviation(SE) net conversion: As the unit of diversion is cookies and net conversion is defined as "The number of user-ids to remain enrolled past the 14-day boundary divided by the number of unique cookies to click the "Start free trial" button.", the unit of denominator of the fraction is also cookies. Because both the unit of diversion and the unit denominator are the same the analytic estimates can be used and it is accurate.

## Sizing

### Number of Samples vs. Power

#### Bonferroni correction

Not used, because the of the high correlation of the metrics and then it will be a to conservative method.

#### Practical significance boundaries ($d_{min}$):

*  Gross conversion $d_{min}$: 0.01
*  Retention $d_{min}$: 0.01
*  Net conversion $d_{min}$: 0.0075

#### Calculate required sample size

In [8]:
def get_z_star(alpha):
    """ Computes the z-critical value for given alpha

    Args:
        alpha

    Returns:
        Z-value
    """
    return -norm.ppf(alpha / 2)

def get_beta(z_star, s, d_min, N):
    """ Computes the Z-value for given alpha

    Args:
        z-star, The z-critical value
        s, The standard error of the metric at N=1
        d_min, The practical significance level
        N, The sample size of each group of the experiment

    Returns:
        beta value of the two-tailed t-test
    """
    
    ### Compute the SE
    SE = s / math.sqrt(N)

    ### Compute and return beta value
    return (norm.cdf(z_star * SE ,loc=d_min,scale=SE))

def clac_sample_size(s, d_min, N=20000, alpha=0.05, beta=0.2):
    """ Computes the needed sample size

    Args:
        z-star, The z-critical value
        s, The standard error of the metric at N=1
        d_min, The practical significance level
        N, The sample size of each group of the experiment

    Returns:
        Smallest number of N to achieve the desired beta. 
        If none found it returns -1.
    """
    
    ### Initalize list generator Ns for given N
    Ns = (x for x in range(1,N,1))
    
    ### Loop through the list of possible N and return N if threshold is meet
    for N in Ns:
        
        ### Compute beta value
        beta_value = get_beta(get_z_star(alpha), s, d_min, N)
        
        ### Check if beta value is smaller as beta
        if beta_value <= beta:

            return N
        
    return -1


### Compute the sample size for probability of enrolling, given click:
ss_gross_conversion = clac_sample_size(s=std_enrolling*math.sqrt(2)*math.sqrt(df_base['ValueSample']['Unique cookies to click "Start free trial" per day:']),
                                       d_min=0.01,
                                       N= 3000000)

### Compute the sample size for probability of payment, given enroll:
ss_retention = clac_sample_size(s=std_payment_enroll*math.sqrt(2)*math.sqrt(df_base['ValueSample']['Enrollments per day:']),
                                       d_min=0.01,
                                       N= 3000000)

### Compute the sample size for probability of payment, given click
ss_net_conversion = clac_sample_size(std_payment_click*math.sqrt(2)*math.sqrt(df_base['ValueSample']['Unique cookies to click "Start free trial" per day:']),
                                       d_min=0.0075,
                                       N= 3000000)

### Print the smallest needed sample size for the evalution metrics
print(colored("Sample size gross conversion: " + str(ss_gross_conversion), 'blue'))
print(colored("Sample size retention: " + str(ss_retention), 'blue'))
print(colored("Sample size net conversion: " + str(ss_net_conversion), 'blue'))

Sample size gross conversion: 25699
Sample size retention: 39104
Sample size net conversion: 27172


#### Calculate required page views

In [9]:
def calc_pageviews(ss, x, y):
    """ Computes smallest number of needed pageviews

    Args:
        ss, smallest number of samples
        x
        y
       
    Returns:
        needed pageviews
    """
    
    return (ss/(y/x))*2

### Compute the pageviews for probability of enrolling, given click:
pv_gross_conversion = calc_pageviews(ss_gross_conversion,
                                     df_base['Value']['Unique cookies to view page per day:'],
                                     df_base['Value']['Unique cookies to click "Start free trial" per day:'])

### Compute the pageviews for probability of payment, given enroll:
pv_retention = calc_pageviews(ss_retention,
                                     df_base['Value']['Unique cookies to view page per day:'],
                                     df_base['Value']['Enrollments per day:'])

### Compute the pageviews for probability of payment, given click
pv_net_conversion = calc_pageviews(ss_net_conversion,
                                     df_base['Value']['Unique cookies to view page per day:'],
                                     df_base['Value']['Unique cookies to click "Start free trial" per day:'])

### Print the smallest needed pageviews for the evalution metrics
print("Page views gross conversion: " + str(pv_gross_conversion))
print("Page views retention: " + str(pv_retention))
print("Page views net conversion: " + str(pv_net_conversion))

Page views gross conversion: 642475.0
Page views retention: 4739878.78788
Page views net conversion: 679300.0


#### Compute total needed pageviews

In [10]:
### Create a list of results
#pv_list = [pv_gross_conversion, pv_retention, pv_net_conversion]
pv_list = [pv_gross_conversion, pv_net_conversion]

### Compute total needed pageviews
pv = max(pv_list)

### Print total needed pageviews
print('Number of needed page views: ' + str(pv))

Number of needed page views: 679300.0


### Duration vs. Exposure

The fraction of traffic is exposed to run this experiment is set to 1. Therefore 50% of the traffic will go in the controll group and 50% in the experiment group.

The reason there in this experiment a fration of 1 is used is, that risk for this experiment is minimal. Because there is no potential harm for the participants as for this experiment cookies are used to identify the users no personal information is collected. Expect in some cases there the user-id is used to distinguish the participants, but even in this case it is just use to distinguish the participants and no further information about the participants is stored and also the participants are not beocome distinguishable through the usage.
Therefore, there is no risk higher than minimal of harm for the participants and the we can use all the traffic.

In [11]:
### Set Fraction
fraction = 1

### Calculate experiment runtime
exp_duration = math.ceil(pv / (df_base['Value']['Unique cookies to view page per day:'] * fraction))

### Print result
print('Duration of the experiment: ' + str(exp_duration) + " days")

Duration of the experiment: 17 days


##### Rationale for removing retention as metric

If we would want to use retention as evalution metric we would have a experiment runtime of 119 days. Therefore, we made the decission as this is experiment runtime is not practial so we decided to not consider etention as evalution metric.

## Experiment Analysis

### Sanity Checks

In [12]:
def conf_interval(controll, experiment, p):
    """ Computes confidence interval for a given experiment and controll group sample size
        and a given probability

    Args:
        controll, number of samples in the controll group
        experiment, number of sample in the experiment group
        p, probability
       
    Returns:
        lower and upper bound of the confidence interval
    """
    ### Estimate the standard deviation
    std = math.sqrt((p * (1-p)) / (controll + experiment))
    
    ### Calculate margin of error
    me = 1.96 * std
    
    ### Return CI
    return (round(p - me, 4), round(p + me, 4))


### Calculate observed value for number of cookies
observed_cookies = round(df_control['Pageviews'].sum() /
                         (df_control['Pageviews'].sum() +
                          df_exp['Pageviews'].sum()), 4)

### Print confidence interval for number of cookies and the observed value
print('CI for number of cookies: '+ 
      colored(str(conf_interval(df_exp['Pageviews'].sum(),df_control['Pageviews'].sum(), 0.5)),'blue')+
      ' Observed: '+
      colored(str(observed_cookies),'blue')+
      colored(' passed', 'blue'))

### Calculate observed value for number of clicks 
observed_user_ids = round(df_control['Clicks'].sum() /
                          (df_control['Clicks'].sum() +
                           df_exp['Clicks'].sum()), 4)

### Print confidence interval for number of clicks and the observed value
print('CI for number of user-ids '+
      colored(str(conf_interval(df_exp['Clicks'].sum(),df_control['Clicks'].sum(), 0.5)),'blue')+
      ' Observed: '+
      colored(str(observed_user_ids),'blue')+
      colored(' passed', 'blue'))

### Calculate observed value for Click-through-probability
observed_ctp = df_exp['Clicks'].sum() / df_exp['Pageviews'].sum()

### Calculate click probaility
click_prob = df_control['Clicks'].sum() / df_control['Pageviews'].sum()

### Print confidence interval for Click-through-probability and the observed value
print('CI for CTP: '+ 
      colored(str(conf_interval(df_exp['Pageviews'].sum(),0, click_prob)),'blue')+
      ' Observed: '+
      colored(str(round(observed_ctp,4)),'blue')+
      colored(' passed', 'blue'))

CI for number of cookies: (0.4988, 0.5012) Observed: 0.5006 passed
CI for number of user-ids (0.4959, 0.5041) Observed: 0.5005 passed
CI for CTP: (0.0812, 0.083) Observed: 0.0822 passed


#### Rationale

For all three invariant metrics the observed value lays within the CI. Therefore, the sanity check is passed for all three metrics.

### Result Analysis

#### Practical significance boundaries ($d_{min}$):

*  Gross conversion $d_{min}$: 0.01
*  Net conversion $d_{min}$: 0.0075

#### Effect Size Tests

In [13]:
def conf_interval_effect(controll_n, controll_x, experiment_n, experiment_x):
    """ Computes confidence interval and the d value

    Args:
        controll_n, number of total samples for the controll group
        controll_x, number of target samples for the controll group
        experiment_n, number of total samples for the experiment group
        experiment_x number of target samples for the experiment group
       
    Returns:
        lower and upper bound of the confidence interval and the d value
    """

    ### Calculate pooled probaiity
    p = (controll_x + experiment_x) / (controll_n + experiment_n)

    ### Calculate pooled SE
    SE = math.sqrt(p * (1-p)*((1/controll_n)+(1/experiment_n)))

    ### Calcualte difference for probaility
    d = (experiment_x/experiment_n) - (controll_x/controll_n)

    ### Calculate margin of error
    me = 1.96 * SE
    
    ### Return CI and d value
    return ((round(d - me, 4), round(d + me, 4)), round(d,4))

### Remove rows with empty Enrollments or Payments column
df_con_no_nan = df_control[['Clicks', 'Enrollments', 'Payments']].dropna()
df_exp_no_nan = df_exp[['Clicks', 'Enrollments', 'Payments']].dropna()

### Calculate CI and d value for gross conversion
effect_gross = conf_interval_effect(df_con_no_nan['Clicks'].sum(),
                                    df_con_no_nan['Enrollments'].sum(),
                                    df_exp_no_nan['Clicks'].sum(),
                                    df_exp_no_nan['Enrollments'].sum())

### Calculate CI and d value for net conversion
effect_net = conf_interval_effect(df_con_no_nan['Clicks'].sum(),
                                  df_con_no_nan['Payments'].sum(),
                                  df_exp_no_nan['Clicks'].sum(),
                                  df_exp_no_nan['Payments'].sum())

### Print confidence interval and d for gross conversion
print('CI gross conversion: ' + 
      colored(str(effect_gross[0]),'blue') + 
      ' d: ' +
      colored(str(effect_gross[1]), 'blue'))

### Print confidence interval and d for net conversion
print('CI net conversion: ' +
      colored(str(effect_net[0]),'blue') + 
      ' d: ' +
      colored(str(effect_net[1]), 'blue'))

CI gross conversion: (-0.0291, -0.012) d: -0.0206
CI net conversion: (-0.0116, 0.0019) d: -0.0049


##### Rationale

Since for gross conversion zero is not included in the CI there is a is statistical significance and also $d_{min}$ and $-d_{min}$ lays not within the CI there is also a practical significance.

Since for net conversion zero is included in the CI there is a is no statistical significance and therefore, also not practical significance.

#### Sign Tests for alpha level of 0.05

In [14]:
### Compute the rate for gross conversion for the
### controll and experiment data and also drop the rows 
### for that no information is available
df_exp['Gross Conversion'] = df_exp['Enrollments']/df_exp['Clicks']
df_exp['Net Conversion'] = df_exp['Payments']/df_exp['Clicks']
df_exp.dropna(inplace = True)

df_control['Gross Conversion'] = df_control['Enrollments']/df_control['Clicks']
df_control['Net Conversion'] = df_control['Payments']/df_control['Clicks']
df_control.dropna(inplace = True)

### Compute the sucess for gross conversion
gc_success = sum(df_exp['Gross Conversion'] - df_control['Gross Conversion'] > 0)

### Compute the sucess for net conversion
nc_success = sum(df_exp['Net Conversion'] - df_control['Net Conversion'] > 0)

### Compute the p-value for gross conversion
p_gross = round(binom_test(gc_success, len(df_exp), 0.5),4)

### Compute the p-value for net conversion
p_net = round(binom_test(nc_success, len(df_exp), 0.5),4)

### Print the p values
print('P-value gross conversion: ' + colored(str(p_gross), 'blue'))
print('P-value net conversion: ' + colored(str(p_net), 'blue'))

P-value gross conversion: 0.0026
P-value net conversion: 0.6776


##### Rationale

As the p-value for gross conversion is less than the alpha value of 0.05 the change is statistical significant.

As the p-value for net conversion is greater than the alpha value of 0.05 the change is not statistical significant.

#### Summary

The Bonferroni correction is not used as as all metrics must be satisfied to trigger launch. All metrics must be satisfied, because it has to be verified that students that get potential frustrated are more unlikely to enroll in the free trial (represented by the gross conversion) and addtionaly it has to be verified that the number of students that are likely to make payments is not reduced (represented by the net conversion).
Therefore, as both of the metrics must be satisfied the Bonferroni correction is not an apropriate choice.

The results of the effect size tests and the sign tests both came to the same result. That the groos conversion is statistically significant reduced and the net conversion rate is not statistically significant improved.

## Recommendation

As a result of the A/B-Test I would recommend to not add the feature, because the test shows that hypothesis related to the feature does not held true.  

The hypothesis relates that the gross conversion must decrease and on the same time the net conversion must increase, because if there is a lower rate of users who enroll in the free trail there have to be a higher fraction of users that stay after the free trail to compensate the users that does not enroll in free trail.

On the one hand side the tests shows that the groos conversion is statistically significant reduced, which represents the users joining the free trial which agrees with the hypothesis.

On the other hand side the tests shows that there is no statistically significant prove for a improve of the net conversion. P-value < alpha. Even if the net conversion doesn't change there is a decrease of users that stays after the free trail. As if the groos conversion decrease the number of enrolled users in the free trail decreases. Therefore, the fraction of users that doesn't leave after the free trial has to be higher, what is represented through the net conversion metric. 

# Follow-Up Experiment

Remove the free trail period. To reduce the number of frustrated students the free trail period is removed. Therefore, the button for start the 14 day free trail period is replaced by a start button that will directly lead to a payed enrollemnt.

Hypothesis:
By removing the free trail period the students are forced to make a decision if they want attend the course before they enrolled, after enrollemnt the students more likely to invest time for this course and the students want not leave the course at the first point they encounter any difficulties. Thus the number of frustrated students is reduced, but at the same time the number of students that directly enroll increases in comparison to the students that make there first payment after the 14 day free trail period.

The invariant metrics are:
*  Number of cookies, number of unique cookies to view the course overview page.


The evalution metrics are:
*  Net conversion, number of user-ids enrolled past the 14-day boundary for the controll group and number of user-ids enrolled without a trail period for the experiment group divided by the number of unique cookies to click the "Start free trial" or the "Start" button. 


The unit of diversion is the a cookie as the the student have not to be logged in to to see the enrollment screen with the .